# Multi-stage feature selection

![End-to-end pipeline](../docs/overview_e2e.jpg)

The multi-stage feature selection method integrates both data science and clinical knowledge to suggest feature candidates for downstream training of proxy models. It consists of multiple stages that are applied in sequential manner. Eventually, the selected features will contain all necessary properties (e.g. high fill rate, dependence with the target) which make them suitable for model training. The following selectors are included:

- `MissingnessSelector` - selecting features based on **fill rate**
- `DataTypeSelector` - selecting features based on **data type**
- `VarianceThreshold` - selecting features based on **variance**
- `RankingSelector` - selecting features based on **ranking method**
    - `SelectFromRF` - selecting based on impurity obtained from shallow [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
    - `SelectFromXGB` - selecting based on impurity obtained from shallow [`XGBRegressor`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor)
    - `SelectFromF_reg` - selecting based on [F-statistic](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html)
    - `SelectFromMutual` - selecting based on [mutual information](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.miual_info_regression.html)

Each method from `RankingSelector` provides a separate list of feature candidates (`TOP_K` % features). We take the union of 4 sets to create the final feature shortlist. 

**Inputs**: 

Please specify `DATA_HOME` in the ["Loading the data models"](#Loading-the-data-models). Put input raw data `AD_final.csv`, `RA_final.csv` there in this folder.

**Outputs**:

- `AD_ppl_final.pkl` - processing pipeline for AD dataset, includes fitted methods for imputation, encoding, normalization
- `RA_ppl_final.pkl` - processing pipeline for RA dataset, includes fitted methods for imputation, encoding, normalization
- `AD_processed_final.csv` - processed AD dataset with features used for training
- `RA_processed_final.csv` - processed RA dataset with features used for training
- `ad_selected_features/ad_multi_stage_features_{TOP_K}.txt` - list of features selected for AD dataset
- `ra_selected_features/ra_multi_stage_features_{TOP_K}.txt` - list of features selected for RA dataset

## Table of contents
    
 - [Multi-stage feature selection](#Multi-stage-feature-selection)<br>
   - [Table of contents](#Table-of-contents)<br>
 - [Preparation](#Preparation)<br>
 - [Loading the data models](#Loading-the-data-models)<br>
 - [Feature dtype identification](#Feature-dtype-identification)<br>
 - [Data preprocessing](#Data-preprocessing)<br>
   - [Preparation](#Preparation)<br>
   - [Splitting data <a id='splitting-data'></a>](#Splitting-data-%3Ca-id%3D%27splitting-data%27%3E%3C/a%3E)<br>
   - [Imputation](#Imputation)<br>
     - [AD](#AD)<br>
     - [RA](#RA)<br>
   - [Category encoding](#Category-encoding)<br>
   - [Normalization](#Normalization)<br>
   - [Saving processed dataframe](#Saving-processed-dataframe)<br>
   - [Getting `train` and `test` data sets as separate dataframes](#Getting-%60train%60-and-%60test%60-data-sets-as-separate-dataframes)<br>
 - [Multi-stage feature selection](#Multi-stage-feature-selection)<br>
   - [`MissingnessSelector`](#%60MissingnessSelector%60)<br>
   - [`DataTypeSelector`](#%60DataTypeSelector%60)<br>
   - [`VarianceThreshold`](#%60VarianceThreshold%60)<br>
   - [`RankingSelector`](#%60RankingSelector%60)<br>
     - [`SelectFromRF`](#%60SelectFromRF%60)<br>
     - [`SelectFromXGB`](#%60SelectFromXGB%60)<br>
     - [`SelectFromF_reg`](#%60SelectFromF_reg%60)<br>
     - [`SelectFromMutual`](#%60SelectFromMutual%60)<br>
 - [Taking the feature union between ranking method outputs](#Taking-the-feature-union-between-ranking-method-outputs)<br>

# Preparation

In [ ]:
# System
import os
import sys
from functools import partial, update_wrapper

import numpy as np

# Data science
import pandas as pd

# Adding the modules to the PYTHONPATH
# add the path to your repository below
REPO_PATH = ""
sys.path.append(REPO_PATH)

from src.utils import *

# Reproducibility
SEED = 42

# Top k % / 100 features to select according to each ranking method
TOP_K = 0.05  # 0.05, 0.01, 0.1

# Loading the data models

In [ ]:
# Add path to clinical trial datasets
DATA_HOME = ""

# Loading the data
# AD_final.csv and RA_final.csv are studies processed according to data engineering methodology within publication
ad_data = pd.read_csv(os.path.join(DATA_HOME, "AD_final.csv"))
ra_data = pd.read_csv(os.path.join(DATA_HOME, "RA_final.csv"))

# Setting up the target, patient ID column names
ad_target_col, ad_patient_id = "endpt_lb_easi1_total_score", "patient_id"
ra_target_col, ra_patient_id = "endpt_lb_das28__crp_", "patient_id"

# Getting the feature sets for both use cases
ad_data = ad_data.rename(columns={"ft_sl_siteid": "site_id"})
ad_all_features = ad_data.filter(regex="^ft_").columns.to_list()
ra_all_features = ra_data.filter(regex="^ft_").columns.to_list()

# Drop records where target is NaN
ad_data = ad_data.dropna(subset=[ad_target_col]).reset_index(drop=True)
ra_data = ra_data.dropna(
    subset=[ra_target_col, "ft_lb_c_reactive_protein__mg_l_"]
).reset_index(drop=True)

# Creating block list of features for both datasets based on
# (1) RWD availability; (2) discussion amongst clinical and technical teams
ad_block_list = [
    "ft_sl_actarm",
    "ft_sl_actarmcd",
    "ft_sl_ageu",
    "ft_sl_arm",
    "ft_sl_armcd",
    "ft_sl_domain",
    "ft_sl_dthdtc",
    "ft_sl_ethnic",
    "ft_sl_invid",
    "ft_sl_invnam",
    "ft_sl_rfstdtc",
    "ft_sl_rfendtc",
    "ft_sl_rficdtc",
    "ft_sl_rfpendtc",
    "ft_sl_rfxendtc",
    "ft_sl_rfxstdtc",
    "ft_sl_rowid",
    "ft_sl_dthfl",
    "ft_sl_subjid",
    "ft_sl_studyid",
] + ad_data.filter(regex="^ft_cc").columns.to_list()
ra_block_list = (
    [
        "ft_sl_actarm",
        "ft_sl_actarmcd",
        "ft_sl_ageu",
        "ft_sl_arm",
        "ft_sl_armcd",
        "ft_sl_domain",
        "ft_sl_dthdtc",
        "ft_sl_dthfl",
        "ft_sl_ethnic",
        "ft_sl_invid",
        "ft_sl_invnam",
        "ft_sl_rfendtc",
        "ft_sl_rficdtc",
        "ft_sl_rfpendtc",
        "ft_sl_rfstdtc",
        "ft_sl_rfxendtc",
        "ft_sl_rfxstdtc",
        "ft_sl_studyid",
        "ft_sl_subjid",
        "ft_sl_siteid",
    ]
    + ra_data.filter(regex="ft_eff").columns.to_list()
    + ["endpt_lb_das28__esr_"]
)

print(
    f"AD dataset contains {ad_data.shape[0]} samples and {ad_data.shape[1]} columns.."
)
print()
print(
    f"RA dataset contains {ra_data.shape[0]} samples and {ra_data.shape[1]} columns.."
)
print()
print("AD dataset:")
display(ad_data.drop(columns=ad_block_list))
print()
print("RA dataset:")
display(ra_data)

# Feature dtype identification

In [ ]:
# No-feature columns
ad_non_features = {ad_target_col, ad_patient_id, "split"}
ra_non_features = {ra_target_col, ra_patient_id, "split"}

# Creating inclusion lists
ad_include_fs = [
    f for f in ad_all_features if f not in ad_block_list and f not in ad_non_features
]
ra_include_fs = [
    f for f in ra_all_features if f not in ra_block_list and f not in ra_non_features
]

# Specifying features based on data type
ad_cat_features = [
    f for f in ad_data.select_dtypes(object).columns.to_list() if f in ad_include_fs
]
ra_cat_features = [
    f for f in ra_data.select_dtypes(object).columns.to_list() if f in ra_include_fs
]

ad_num_features = [f for f in ad_include_fs if f not in ad_cat_features]
ra_num_features = [f for f in ra_include_fs if f not in ra_cat_features]

# Creating final feature sets
ad_features = set(ad_num_features) | set(
    ad_cat_features
)  # not including dt features now in `features`

ra_features = set(ra_num_features) | set(
    ra_cat_features
)  # not including dt features now in `features`

# Data preprocessing

## Preparation

Let's write a method-agnostic function that will preprocess input data with `scikit-learn` API.

In [ ]:
def apply_fit_transform(transformers, train_test_data, target_col):
    """Fit and transform data based on passed list of transformers."""

    processed_data = train_test_data.copy()

    for tag, transformer, features in transformers:
        transformer.fit(
            processed_data.loc[processed_data["split"] == "TRAIN", features],
            processed_data.loc[processed_data["split"] == "TRAIN", target_col],
        )
        processed_data[features] = transformer.transform(processed_data[features])

    return processed_data


ad_ppl, ra_ppl = {}, {}

## Splitting data <a id='splitting-data'></a>

Here we will split the original data into `train` and `test` sets by adding a column `"split"`.

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold


def add_stratification_col(_data, _target):
    """Add a column that will stratify the target distribution by binning into 5 buckets."""
    bins = _data[_target].quantile([0, 0.25, 0.5, 0.75, 1]).to_numpy()
    bins[-1] += 1
    inds = np.digitize(_data[_target], bins)
    _data["stratification_col"] = [
        f"{bins[inds[i]-1]:.2f} <= {_target} < {bins[inds[i]]:.2f}"
        for i in range(_data.shape[0])
    ]

    return _data


def make_splits(data, splitter_func, group_col_name, target_col_name):
    """Splits data into TRAIN and TEST with a passed splitter."""

    split_func_args = {
        "y": data[target_col_name].copy(),
        "groups": data[group_col_name].copy(),
    }

    indices = list(splitter_func.split(data, **split_func_args))
    train_index, test_index = indices[0]
    data.loc[train_index, "split"] = "TRAIN"
    data.loc[test_index, "split"] = "TEST"

    return data


test_size = 0.1

# AD
# Separating data into train and test sets
ad_data = add_stratification_col(ad_data, ad_target_col)
ad_train_test_data = make_splits(
    data=ad_data,
    splitter_func=StratifiedGroupKFold(
        n_splits=int(1 / test_size), random_state=SEED, shuffle=True
    ),
    target_col_name="stratification_col",
    group_col_name=ad_patient_id,
)

print("AD dataset:")
print(
    f'   Training set: {ad_train_test_data.loc[lambda x: x["split"] == "TRAIN"].shape[0]} samples'
)
print(
    f'   Testing set: {ad_train_test_data.loc[lambda x: x["split"] == "TEST"].shape[0]} samples'
)
print()

# RA
# Separating data into train and test sets
ra_data = add_stratification_col(ra_data, ra_target_col)
ra_train_test_data = make_splits(
    data=ra_data,
    splitter_func=StratifiedGroupKFold(
        n_splits=int(1 / test_size), random_state=SEED, shuffle=True
    ),
    target_col_name="stratification_col",
    group_col_name=ra_patient_id,
)

print("RA dataset:")
print(
    f'   Training set: {ra_train_test_data.loc[lambda x: x["split"] == "TRAIN"].shape[0]} samples'
)
print(
    f'   Testing set: {ra_train_test_data.loc[lambda x: x["split"] == "TEST"].shape[0]} samples'
)

## Imputation

We will start by removing features with extremely high missingness (>90 %).

### AD

In [ ]:
from sklearn.impute import SimpleImputer

# A percentage threshold to flag the features that will be dropped due to high level of missingness, must be a list (with at least 1 element)
NA_THRESH = 0.9

# Define feature set where imputation will be skipped according to `NA_THRESH`
ad_na_skip_features = set(
    (ad_train_test_data.isna().sum() / ad_train_test_data.shape[0])
    .loc[lambda x: x > NA_THRESH]
    .index
)


# Defining pipeline options
ad_imputation_params = [
    (
        "imp_Simple_cats",
        SimpleImputer(strategy="most_frequent"),
        list(set(ad_cat_features) - set(ad_na_skip_features)),
    ),
    (
        "imp_Simple_nums",
        SimpleImputer(strategy="median"),
        list(set(ad_num_features) - set(ad_na_skip_features)),
    ),
]

print(
    f"The following features will be skipped from the analysis due to missingness (if not saved by `free_pass_list`):"
)
print(f"   NA threshold = {NA_THRESH}: {ad_na_skip_features}")
print(ad_imputation_params)

ad_imp_data = apply_fit_transform(
    ad_imputation_params, ad_train_test_data, ad_target_col
)

ad_ppl["imp"] = ad_imputation_params

### RA

In [ ]:
from sklearn.impute import SimpleImputer

# A percentage threshold to flag the features that will be dropped due to high level of missingness, must be a list (with at least 1 element)
NA_THRESH = 0.9

# Define feature set where imputation will be skipped according to `NA_THRESH`
ra_na_skip_features = set(
    (ra_train_test_data.isna().sum() / ra_train_test_data.shape[0])
    .loc[lambda x: x > NA_THRESH]
    .index
)


# Defining pipeline options
ra_imputation_params = [
    (
        "imp_Simple_cats",
        SimpleImputer(strategy="most_frequent"),
        list(set(ra_cat_features) - set(ra_na_skip_features)),
    ),
    (
        "imp_Simple_nums",
        SimpleImputer(strategy="median"),
        list(set(ra_num_features) - set(ra_na_skip_features)),
    ),
]

print(
    f"The following features will be skipped from the analysis due to missingness (if not saved by `free_pass_list`):"
)
print(f"   NA threshold = {NA_THRESH}: {ra_na_skip_features}")
print(ra_imputation_params)

ra_imp_data = apply_fit_transform(
    ra_imputation_params, ra_train_test_data, ra_target_col
)
ra_ppl["imp"] = ra_imputation_params

## Category encoding

In [ ]:
from category_encoders.target_encoder import TargetEncoder

# AD
print("AD dataset:")
ad_category_encoding_params = [
    (
        "cat_enc_Target",
        TargetEncoder(),
        list(set(ad_cat_features) - set(ad_na_skip_features)),
    )
]
print(ad_category_encoding_params)
ad_cat_enc_data = apply_fit_transform(
    ad_category_encoding_params, ad_imp_data, ad_target_col
)
ad_ppl["cat_enc"] = ad_category_encoding_params
print()

# RA
print("RA dataset:")
ra_category_encoding_params = [
    (
        "cat_enc_Target",
        TargetEncoder(),
        list(set(ra_cat_features) - set(ra_na_skip_features)),
    )
]
ra_cat_enc_data = apply_fit_transform(
    ra_category_encoding_params, ra_imp_data, ra_target_col
)
ra_ppl["cat_enc"] = ra_category_encoding_params
print(ra_category_encoding_params)

## Normalization

In [ ]:
from sklearn.preprocessing import RobustScaler

# AD
print("AD dataset:")
ad_normalization_params = [
    (
        "norm_Robust",
        RobustScaler(),
        list(set(ad_num_features) - set(ad_na_skip_features)),
    )
]
print(ad_normalization_params)
ad_norm_data = apply_fit_transform(
    ad_normalization_params, ad_cat_enc_data, ad_target_col
)
ad_ppl["norm"] = ad_normalization_params
print()
# RA
print("RA dataset:")
ra_normalization_params = [
    (
        "norm_Robust",
        RobustScaler(),
        list(set(ra_num_features) - set(ra_na_skip_features)),
    )
]
print(ra_normalization_params)
ra_norm_data = apply_fit_transform(
    ra_normalization_params, ra_cat_enc_data, ra_target_col
)
ra_ppl["norm"] = ra_normalization_params

## Saving processed dataframe

In [ ]:
fn = os.path.join(DATA_HOME, "AD_processed_final.csv")
if os.path.exists(fn):
    raise ValueError(f"{fn} already exists in the data folder.")
else:
    ad_norm_data.to_csv(fn, index=False)

fn = os.path.join(DATA_HOME, "RA_processed_final.csv")
if os.path.exists(fn):
    raise ValueError(f"{fn} already exists in the data folder.")
else:
    ra_norm_data.to_csv(fn, index=False)

write_pickle(ad_ppl, os.path.join(DATA_HOME, "AD_ppl_final.pkl"))
write_pickle(ra_ppl, os.path.join(DATA_HOME, "RA_ppl_final.pkl"))

## Getting `train` and `test` data sets as separate dataframes

In [ ]:
ad_features_to_use = [f for f in ad_features if f not in ad_na_skip_features]
ra_features_to_use = [f for f in ra_features if f not in ra_na_skip_features]

# Getting `X`s
ad_X_train, ad_X_test = (
    ad_norm_data.loc[ad_norm_data["split"] == "TRAIN", ad_features_to_use],
    ad_norm_data.loc[ad_norm_data["split"] == "TEST", ad_features_to_use],
)
ra_X_train, ra_X_test = (
    ra_norm_data.loc[ra_norm_data["split"] == "TRAIN", ra_features_to_use],
    ra_norm_data.loc[ra_norm_data["split"] == "TEST", ra_features_to_use],
)

# Getting `groups`s
ad_groups_train, ad_groups_test = (
    ad_norm_data.loc[ad_norm_data["split"] == "TRAIN", ad_patient_id],
    ad_norm_data.loc[ad_norm_data["split"] == "TEST", ad_patient_id],
)
ra_groups_train, ra_groups_test = (
    ra_norm_data.loc[ra_norm_data["split"] == "TRAIN", ra_patient_id],
    ra_norm_data.loc[ra_norm_data["split"] == "TEST", ra_patient_id],
)

# Creating `y`s
ad_y_train, ad_y_test = (
    ad_norm_data.loc[ad_norm_data["split"] == "TRAIN", ad_target_col],
    ad_norm_data.loc[ad_norm_data["split"] == "TEST", ad_target_col],
)
ra_y_train, ra_y_test = (
    ra_norm_data.loc[ra_norm_data["split"] == "TRAIN", ra_target_col],
    ra_norm_data.loc[ra_norm_data["split"] == "TEST", ra_target_col],
)

print(
    f"AD dataset contains:\n\t"
    f"TRAIN: {ad_X_train.shape[1]} features and {ad_X_train.shape[0]} samples!\n\t"
    f"TEST: {ad_X_test.shape[1]} features and {ad_X_test.shape[0]} samples!\n\t"
)
print(
    f"RA dataset contains:\n\t"
    f"TRAIN: {ra_X_train.shape[1]} features and {ra_X_train.shape[0]} samples!\n\t"
    f"TEST: {ra_X_test.shape[1]} features and {ra_X_test.shape[0]} samples!\n\t"
)

# Multi-stage feature selection

Let us implement each stage separately and then we will apply them sequentially and take the union at the end.

## `MissingnessSelector`

We will first drop all features that have fill rate less than 50 %. We need to calculate it on the original data.

In [ ]:
# A percentage threshold to flag the features that will be dropped due to high level of missingness, must be a list (with at least 1 element)
MISSING_THRESH = 0.5

# Define feature set where imputation will be skipped according to `MISSING_THRESH`
ad_features_to_drop = set(
    (ad_train_test_data.isna().sum() / ad_train_test_data.shape[0])
    .loc[lambda x: x > MISSING_THRESH]
    .index
)

# Define feature set where imputation will be skipped according to `MISSING_THRESH`
ra_features_to_drop = set(
    (ra_train_test_data.isna().sum() / ra_train_test_data.shape[0])
    .loc[lambda x: x > MISSING_THRESH]
    .index
)

# Dropping features
ad_X_train_present, ad_X_test_present = (
    ad_X_train.drop(columns=ad_features_to_drop, errors="ignore"),
    ad_X_test.drop(columns=ad_features_to_drop, errors="ignore"),
)
ra_X_train_present, ra_X_test_present = (
    ra_X_train.drop(columns=ra_features_to_drop, errors="ignore"),
    ra_X_test.drop(columns=ra_features_to_drop, errors="ignore"),
)

print(
    f"AD: the number of features went from {ad_X_train.shape[1]} to {ad_X_train_present.shape[1]}"
)
print(
    f"RA: the number of features went from {ra_X_train.shape[1]} to {ra_X_train_present.shape[1]}"
)

## `DataTypeSelector`

Let us select only numerical features as selected ML algorithms only handle numerical values.

In [ ]:
# Selecting only float numbers
ad_X_train_num, ad_X_test_num = (
    ad_X_train_present.select_dtypes(float),
    ad_X_test_present.select_dtypes(float),
)
ra_X_train_num, ra_X_test_num = (
    ra_X_train_present.select_dtypes(float),
    ra_X_test_present.select_dtypes(float),
)

print(
    f"AD: the number of features went from {ad_X_train_present.shape[1]} to {ad_X_train_num.shape[1]}"
)
print(
    f"RA: the number of features went from {ra_X_train_present.shape[1]} to {ra_X_train_num.shape[1]}"
)

## `VarianceThreshold`

Let us select only features with high-enough variance.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

VAR_THRESH = 0.1

# AD
var_selector = VarianceThreshold()
var_selector.fit(ad_X_train_num)
ad_X_train_var, ad_X_test_var = (
    pd.DataFrame(
        var_selector.transform(ad_X_train_num),
        columns=var_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        var_selector.transform(ad_X_test_num),
        columns=var_selector.get_feature_names_out(),
    ),
)

# # RA
var_selector = VarianceThreshold()
var_selector.fit(ra_X_train_num)
ra_X_train_var, ra_X_test_var = (
    pd.DataFrame(
        var_selector.transform(ra_X_train_num),
        columns=var_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        var_selector.transform(ra_X_test_num),
        columns=var_selector.get_feature_names_out(),
    ),
)

print(
    f"AD: the number of features went from {ad_X_train_num.shape[1]} to {ad_X_train_var.shape[1]}"
)
print(
    f"RA: the number of features went from {ra_X_train_num.shape[1]} to {ra_X_train_var.shape[1]}"
)

## `RankingSelector`

Now we are ready to apply ranking selectors and take the feature union based on 4 outputs. Each method will select the top 5% of features based on its ranking.

### `SelectFromRF`

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

# AD
ad_top_k = int(np.ceil(TOP_K * ad_X_train_var.shape[1]))
rf_selector = SelectFromModel(
    RandomForestRegressor(n_estimators=10, random_state=SEED),
    threshold=1e-5,
    importance_getter=partial(importance_getter_k, k=ad_top_k),
)
rf_selector.fit(ad_X_train_var, ad_y_train)
ad_X_train_rf, ad_X_test_rf = (
    pd.DataFrame(
        rf_selector.transform(ad_X_train_var),
        columns=rf_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        rf_selector.transform(ad_X_test_var),
        columns=rf_selector.get_feature_names_out(),
    ),
)

# # RA
ra_top_k = int(np.ceil(TOP_K * ra_X_train_var.shape[1]))
rf_selector = SelectFromModel(
    RandomForestRegressor(n_estimators=10, random_state=SEED),
    threshold=1e-5,
    importance_getter=partial(importance_getter_k, k=ra_top_k),
)
rf_selector.fit(ra_X_train_var, ra_y_train)
ra_X_train_rf, ra_X_test_rf = (
    pd.DataFrame(
        rf_selector.transform(ra_X_train_var),
        columns=rf_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        rf_selector.transform(ra_X_test_var),
        columns=rf_selector.get_feature_names_out(),
    ),
)

print(
    f"AD: the number of features went from {ad_X_train_var.shape[1]} to {ad_X_train_rf.shape[1]}"
)
print(
    f"RA: the number of features went from {ra_X_train_var.shape[1]} to {ra_X_train_rf.shape[1]}"
)

### `SelectFromXGB`

In [ ]:
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor

# AD
ad_top_k = int(np.ceil(TOP_K * ad_X_train_var.shape[1]))
xgb_selector = SelectFromModel(
    XGBRegressor(n_estimators=10, random_state=SEED),
    threshold=1e-5,
    importance_getter=partial(importance_getter_k, k=ad_top_k),
)
xgb_selector.fit(ad_X_train_var, ad_y_train)
ad_X_train_xgb, ad_X_test_xgb = (
    pd.DataFrame(
        xgb_selector.transform(ad_X_train_var),
        columns=xgb_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        xgb_selector.transform(ad_X_test_var),
        columns=xgb_selector.get_feature_names_out(),
    ),
)

# RA
ra_top_k = int(np.ceil(TOP_K * ra_X_train_var.shape[1]))
xgb_selector = SelectFromModel(
    XGBRegressor(n_estimators=10, random_state=SEED),
    threshold=1e-5,
    importance_getter=partial(importance_getter_k, k=ra_top_k),
)
xgb_selector.fit(ra_X_train_var, ra_y_train)
ra_X_train_xgb, ra_X_test_xgb = (
    pd.DataFrame(
        xgb_selector.transform(ra_X_train_var),
        columns=xgb_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        xgb_selector.transform(ra_X_test_var),
        columns=xgb_selector.get_feature_names_out(),
    ),
)

print(
    f"AD: the number of features went from {ad_X_train_var.shape[1]} to {ad_X_train_xgb.shape[1]}"
)
print(
    f"RA: the number of features went from {ra_X_train_var.shape[1]} to {ra_X_train_xgb.shape[1]}"
)

### `SelectFromF_reg`

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# AD
ad_top_k = int(np.ceil(TOP_K * ad_X_train_var.shape[1]))
f_selector = SelectKBest(f_regression, k=ad_top_k)
f_selector.fit(ad_X_train_var, ad_y_train)
ad_X_train_f, ad_X_test_f = (
    pd.DataFrame(
        f_selector.transform(ad_X_train_var), columns=f_selector.get_feature_names_out()
    ),
    pd.DataFrame(
        f_selector.transform(ad_X_test_var), columns=f_selector.get_feature_names_out()
    ),
)

# RA
ra_top_k = int(np.ceil(TOP_K * ra_X_train_var.shape[1]))
f_selector = SelectKBest(f_regression, k=ra_top_k)
f_selector.fit(ra_X_train_var, ra_y_train)
ra_X_train_f, ra_X_test_f = (
    pd.DataFrame(
        f_selector.transform(ra_X_train_var), columns=f_selector.get_feature_names_out()
    ),
    pd.DataFrame(
        f_selector.transform(ra_X_test_var), columns=f_selector.get_feature_names_out()
    ),
)

print(
    f"AD: the number of features went from {ad_X_train_var.shape[1]} to {ad_X_train_f.shape[1]}"
)
print(
    f"RA: the number of features went from {ra_X_train_var.shape[1]} to {ra_X_train_f.shape[1]}"
)

### `SelectFromMutual`

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

# AD
ad_top_k = int(np.ceil(TOP_K * ad_X_train_var.shape[1]))
fixed_seed_mi = update_wrapper(
    partial(mutual_info_regression, random_state=SEED), mutual_info_regression
)
mi_selector = SelectKBest(fixed_seed_mi, k=ad_top_k)
mi_selector.fit(ad_X_train_var, ad_y_train)
ad_X_train_mi, ad_X_test_mi = (
    pd.DataFrame(
        mi_selector.transform(ad_X_train_var),
        columns=mi_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        mi_selector.transform(ad_X_test_var),
        columns=mi_selector.get_feature_names_out(),
    ),
)

# RA
ra_top_k = int(np.ceil(TOP_K * ra_X_train_var.shape[1]))
fixed_seed_mi = update_wrapper(
    partial(mutual_info_regression, random_state=SEED), mutual_info_regression
)
mi_selector = SelectKBest(fixed_seed_mi, k=ra_top_k)
mi_selector.fit(ra_X_train_var, ra_y_train)
ra_X_train_mi, ra_X_test_mi = (
    pd.DataFrame(
        mi_selector.transform(ra_X_train_var),
        columns=mi_selector.get_feature_names_out(),
    ),
    pd.DataFrame(
        mi_selector.transform(ra_X_test_var),
        columns=mi_selector.get_feature_names_out(),
    ),
)

print(
    f"AD: the number of features went from {ad_X_train_var.shape[1]} to {ad_X_train_mi.shape[1]}"
)
print(
    f"RA: the number of features went from {ra_X_train_var.shape[1]} to {ra_X_train_mi.shape[1]}"
)

# Taking the feature union between ranking method outputs

In [ ]:
# AD
ad_multi_stage_features = list(
    set().union(
        *[
            ad_X_train_rf.columns.to_list(),
            ad_X_train_xgb.columns.to_list(),
            ad_X_train_f.columns.to_list(),
            ad_X_train_mi.columns.to_list(),
        ]
    )
)
print("AD features:")
for pref in ["ft_sl", "ft_lb", "ft_mh", "ft_cm"]:
    curr_fts = [
        t.replace(f"{pref}_", "") for t in ad_multi_stage_features if t.startswith(pref)
    ]
    print(f"{pref} ({len(curr_fts)}): {', '.join(curr_fts)}")
    print()

# Save the features to a file in DATA_HOME/ad_selected_features
fn = os.path.join(
    DATA_HOME, "ad_selected_features", f"ad_multi_stage_features_{TOP_K}.txt"
)
write_list(ad_multi_stage_features, fn)

# RA
ra_multi_stage_features = list(
    set().union(
        *[
            ra_X_train_rf.columns.to_list(),
            ra_X_train_xgb.columns.to_list(),
            ra_X_train_f.columns.to_list(),
            ra_X_train_mi.columns.to_list(),
        ]
    )
)
print()
print()
print("RA features:")
for pref in ["ft_sl", "ft_lb", "ft_mh", "ft_cm"]:
    curr_fts = [
        t.replace(f"{pref}_", "") for t in ra_multi_stage_features if t.startswith(pref)
    ]
    print(f"{pref} ({len(curr_fts)}): {', '.join(curr_fts)}")
    print()
# Save the features to a file in DATA_HOME/ra_selected_features
fn = os.path.join(
    DATA_HOME, "ra_selected_features", f"ra_multi_stage_features_{TOP_K}.txt"
)
write_list(ra_multi_stage_features, fn)

In [ ]:
def calculate_number_in_each_group(features, name):
    return pd.Series(
        {
            "sl": len([f for f in features if f.startswith("ft_sl")]),
            "lb": len([f for f in features if f.startswith("ft_lb")]),
            "mh": len([f for f in features if f.startswith("ft_mh")]),
            "cm": len([f for f in features if f.startswith("ft_cm")]),
        },
        name=name,
    )


out = (
    pd.DataFrame(columns=["sl", "lb", "mh", "cm"])
    .append(calculate_number_in_each_group(ad_X_train.columns, "Initial Set"))
    .append(
        calculate_number_in_each_group(
            ad_X_train_present.columns, "Availability Threshold"
        )
    )
    .append(calculate_number_in_each_group(ad_X_train_num.columns, "Data Type Filter"))
    .append(
        calculate_number_in_each_group(ad_X_train_var.columns, "Variance Threshold")
    )
    .append(calculate_number_in_each_group(ad_X_train_mi.columns, "Mutual Information"))
    .append(calculate_number_in_each_group(ad_X_train_f.columns, "F-Test"))
    .append(calculate_number_in_each_group(ad_X_train_rf.columns, "Random Forest"))
    .append(calculate_number_in_each_group(ad_X_train_xgb.columns, "XGBoost"))
    .append(calculate_number_in_each_group(ad_multi_stage_features, "Multi-Stage"))
)
out

In [ ]:
def calculate_number_in_each_group(features, name):
    return pd.Series(
        {
            "sl": len([f for f in features if f.startswith("ft_sl")]),
            "lb": len([f for f in features if f.startswith("ft_lb")]),
            "mh": len([f for f in features if f.startswith("ft_mh")]),
            "cm": len([f for f in features if f.startswith("ft_cm")]),
        },
        name=name,
    )


out = (
    pd.DataFrame(columns=["sl", "lb", "mh", "cm"])
    .append(calculate_number_in_each_group(ra_X_train.columns, "Initial Set"))
    .append(
        calculate_number_in_each_group(
            ra_X_train_present.columns, "Availability Threshold"
        )
    )
    .append(calculate_number_in_each_group(ra_X_train_num.columns, "Data Type Filter"))
    .append(
        calculate_number_in_each_group(ra_X_train_var.columns, "Variance Threshold")
    )
    .append(calculate_number_in_each_group(ra_X_train_mi.columns, "Mutual Information"))
    .append(calculate_number_in_each_group(ra_X_train_f.columns, "F-Test"))
    .append(calculate_number_in_each_group(ra_X_train_rf.columns, "Random Forest"))
    .append(calculate_number_in_each_group(ra_X_train_xgb.columns, "XGBoost"))
    .append(calculate_number_in_each_group(ra_multi_stage_features, "Multi-Stage"))
)
out